# 0. Now we have chose FoodVision from PyTorvh torchvision to work with

We need to:
1. Prepare our data from torch vision
2. Build a model( we will use pre-build model to make comparison)
    * 1. Choose an optimizer and loss function
    * 2. Design a training and testing loop
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through experiment
6. Save model

## 3. Fit the model to the data and make a prediction

In [2]:
## Setupo image path for train and testing

from pathlib import Path

image_path = Path("data/")
image_data = image_path / "food_10_percent"

train_dir = image_data /  "train"
test_dir = image_data / "test"

train_dir,test_dir

(WindowsPath('data/food_10_percent/train'),
 WindowsPath('data/food_10_percent/test'))

# Step for fiiting data into model
* image path -> image -> image tensor ->  dataloader

In [3]:
from torchvision import transforms

# Create transformer for the image
simple_transforms = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])
train_transforms = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
simple_transforms, train_transforms

(Compose(
     Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
     ToTensor()
 ),
 Compose(
     Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
     TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
     ToTensor()
 ))

In [4]:
# create dataloader for training and testing loop
from torchvision import datasets
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 16
NUM_WORKER = os.cpu_count()

# Convert image into datasets
train_datasets_20 = datasets.ImageFolder(root=train_dir,
                                           transform=train_transforms)
test_datasets_20 = datasets.ImageFolder(root=test_dir,
                                          transform=simple_transforms)

# get class name from here
classes = train_datasets_20.classes

# Convert datasets into dataloader
train_dataloader_20 = DataLoader(dataset=train_datasets_20,
                                 batch_size=BATCH_SIZE,
                                 shuffle=True,
                                 num_workers=NUM_WORKER,
                                 pin_memory=True)

test_dataloader_20 = DataLoader(dataset=test_datasets_20,
                                 batch_size=BATCH_SIZE,
                                 shuffle=False,
                                 num_workers=NUM_WORKER,
                                 pin_memory=True)
train_dataloader_20, test_dataloader_20, len(classes)

(<torch.utils.data.dataloader.DataLoader at 0x1b736942dd0>,
 101)

code is done let's put all of them into a function and translate into python

In [5]:
# create dataloader for training and testing loop
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision
import os

def create_dataloaders(train_dir:str,
               test_dir:str,
               train_transforms: torchvision.transforms.Compose,
               test_transforms: torchvision.transforms.Compose,
               batch_size: int,
               num_worker: int =os.cpu_count()):
    
    # convert image from path into datsets
    train_datasets = datasets.ImageFolder(root=train_dir,
                                          transform=train_transforms,
                                          )
    test_datasets = datasets.ImageFolder(root=test_dir,
                                         transform= test_transforms)
    
    # Get classes
    classes = train_datasets.classes
    
    # convert datsets into dataloader
    train_dataloader = DataLoader(train_datasets,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_worker,
                                  pin_memory=True)
    
    test_dataloader = DataLoader(test_datasets,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_worker,
                                 pin_memory=True)
    
    return train_dataloader, test_dataloader, classes

Now data has been set

Let's train our model

In [6]:
from scripts import data_setup, engine, model
import torch as T
import torch.nn as nn
BATCH_SIZE = 32
#
device = "cuda" if T.cuda.is_available() else "cpu"

train_dataloader_20, test_dataloader_20, classes = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                 test_dir=test_dir,
                                                                                 train_transforms=train_transforms,
                                                                                 test_transforms=simple_transforms,
                                                                                 batch_size=BATCH_SIZE)

#train_dataloader_20, test_dataloader_20, len(classes)

# create model
modelV1 = model.VGGV0(input_size=3,
                    hidden_units=32,
                    output_size=len(classes)).to(device)

#modelV1
# setup optimizer and loss fn

loss_fn = nn.CrossEntropyLoss()
optimizer = T.optim.Adam(params=modelV1.parameters(),
                         lr=0.01)

# start training

results =engine.train(model=modelV1,
                    train_dataloader = train_dataloader_20,
                    test_dataloader = test_dataloader_20,
                    loss_fn=loss_fn,
                    optimizer=optimizer,
                    epochs=5,
                    device=device)


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 4.6206 | train_acc: 0.0095 | test_loss: 4.6181 | test_acc: 0.0091
Epoch: 2 | train_loss: 4.6174 | train_acc: 0.0098 | test_loss: 4.6176 | test_acc: 0.0119
Epoch: 3 | train_loss: 4.6174 | train_acc: 0.0103 | test_loss: 4.6182 | test_acc: 0.0087
Epoch: 4 | train_loss: 4.6170 | train_acc: 0.0104 | test_loss: 4.6175 | test_acc: 0.0087
Epoch: 5 | train_loss: 4.6172 | train_acc: 0.0103 | test_loss: 4.6179 | test_acc: 0.0087


## 5. Evaluate model

In [12]:
test_loss, test_acc = engine.test_step(
    modelV1,test_dataloader_20,loss_fn,device
)
test_loss,test_acc

(4.617881008341342, 0.00870253164556962)